In [104]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import os
import glob
import fastai

In [105]:
from pathlib import Path
from fastai import *
from fastai.vision import *
import torch
from fastai.callback import *
from tqdm import tqdm_notebook

# more precise importing the required libraries
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16,ResNet101,ResNet101V2
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split

from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

from sklearn.preprocessing import LabelEncoder

In [106]:
img_folder = Path("../Snek")
train_path = "../Snek/train"
path = os.path.join(train_path , "*jpg")

In [107]:
files = glob.glob(path)
data = []
name_img_mapper = {}
for file in tqdm_notebook(files):
    fn = file.split('/')[-1]
    image = cv2.imread(file)
    data.append(file)
    name_img_mapper[fn] = image

<ipython-input-107-6ae664c70376>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for file in tqdm_notebook(files):


  0%|          | 0/5508 [00:00<?, ?it/s]

In [108]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [109]:
#Concatenate .jpg to simplify
train = train_df.copy()
test = test_df.copy()

train['image_id'] = train['image_id']+str('.jpg')
test['image_id'] = test['image_id']+str('.jpg')

print(train.head)
print(test.head)

<bound method NDFrame.head of             image_id                        breed
0     a8b3ad1dde.jpg        nerodia-erythrogaster
1     8b492b973d.jpg        pantherophis-vulpinus
2     929b99ea92.jpg          thamnophis-sirtalis
3     bbac7385e2.jpg       pantherophis-obsoletus
4     ef776b1488.jpg       agkistrodon-contortrix
...              ...                          ...
5503  5f0c970adf.jpg      lampropeltis-triangulum
5504  f6a4287fac.jpg  pantherophis-alleghaniensis
5505  d3c10e63e1.jpg          thamnophis-sirtalis
5506  61ef50879c.jpg          pituophis-catenifer
5507  4791d8bf9a.jpg      lampropeltis-triangulum

[5508 rows x 2 columns]>
<bound method NDFrame.head of             image_id
0     7ede553357.jpg
1     3500b219e4.jpg
2     d43a78d6d9.jpg
3     1f36f26994.jpg
4     8cb85fc58e.jpg
...              ...
2356  93e591cb23.jpg
2357  68a603b39d.jpg
2358  4f472510f3.jpg
2359  d6884444ae.jpg
2360  c0fa7df5de.jpg

[2361 rows x 1 columns]>


In [110]:
train.groupby('breed').count()

,image_id
breed,
agkistrodon-contortrix,206
agkistrodon-piscivorus,147
coluber-constrictor,129
crotalus-atrox,354
crotalus-horridus,142
crotalus-ruber,77
crotalus-scutulatus,63
crotalus-viridis,83
diadophis-punctatus,184


In [111]:
sns.countplot(x='breed', data=train)

<AxesSubplot:xlabel='breed', ylabel='count'>

In [112]:
train_images = data

In [113]:
le = LabelEncoder()
training_labels = le.fit_transform(train['breed'])

training_labels

array([15, 25, 34, ..., 34, 26, 12])

In [114]:
#display image samples
train.loc[train['breed']== 'pantherophis-spiloides']['image_id'][:3].tolist()

['299c932a68.jpg', '45652aebed.jpg', '2bc6a0b4cd.jpg']

In [115]:
def plot_class(cat):

    fetch = train.loc[train_df['breed']== cat][:3]
    image_names = train.loc[train_df['breed']== cat]['image_id'] [:3] .tolist()

    fig = plt.figure(figsize=(20,15))

    for i, img_name in enumerate(image_names):
        plt.subplot(1,3 ,i+1)
        plt.imshow(name_img_mapper[img_name])
    plt.show()

In [1]:
plt.imshow(name_img_mapper['299c932a68.jpg'])

NameError: name 'plt' is not defined

In [ ]:
plot_class('pantherophis-spiloides')

In [ ]:
#Transforming images
tfms = get_transforms(do_flip=True, flip_vert=False ,max_rotate=20.0, max_zoom=1.3, max_lightning=0.5,
                      max_warp=0.1, p_affine=0.2, p_lightning=0.55)

#list from test set
test_img = Imagelist.from_df(test, path=data_folder, folder='test')

In [ ]:
np.random.seed(145)

src = (Imagelist.from_df(train, path=data_folder, folder='train').split_by_rand_pct(0.10)
       .label_from_df
       .add_test(test_img))